# 트리 구조 기반 챗봇 만들기

In [ ]:
!pip install sentence_transformers

import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b4ceace0d8fc01f4d3061a534e60460806fbbc08d309c0721e5b9da24941fcde
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


# 역할별 학습 모델 로딩

In [ ]:
%cd /content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/tree_chatbot

# 현재 단계별로 학습된 모델이라고 가정

# 의도 분류 모델
model1 = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 예약 정보 확인 모델
model2 = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 재예약 여부 확인 모델
model3 = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/tree_chatbot


In [ ]:
# CNN_reserve_info (예약 정보) 모델 로딩
!pip install konlpy
from konlpy.tag import Komoran
tokenizer = Komoran()

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
import pickle

class Preprocess:
    def __init__(self, word2index_dic='/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/love_chatbot_sbert_v2(custom_data)/data/chatbot_dict/chatbot_dict.bin' ,userdic=None): # userdic 인자에는 사용자 정의 사전 파일 경로 입력가능
        # 단어 인덱스 사전 불러오기 추가
        if(word2index_dic != ''):
            f = open(word2index_dic, "rb")
            self.word_index = pickle.load(f)
            f.close()
            print("단어 사전 로드 완료..")
        else:
            self.word_index = None
            print("단어 사전 로드 실패..")

        # 형태소 분석기 초기화
        self.komoran = Komoran(userdic=userdic)

        # 제외할 품사
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언 제거, 기호 제거
        # 어미 제거
        # 접미사 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            # 주격조사, 보격조사, 관형격조사, 목적격조사, 부사격조사, 호격조사, 인용격조사
            'JX', 'JC',
            # 보조사, 접속조사
            'SF', 'SP', 'SS', 'SE', 'SO',
            # 마침표,물음표,느낌표(SF), 쉼표,가운뎃점,콜론,빗금(SP), 따옴표,괄호표,줄표(SS), 줄임표(SE), 붙임표(물결,숨김,빠짐)(SO)
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            # 선어말어미, 종결어미, 연결어미, 명사형전성어미, 관형형전성어미
            'XSN', 'XSV', 'XSA'
            # 명사파생접미사, 동사파생접미사, 형용사파생접미사
        ]


    # 형태소 분석기 POS 태거
    def pos(self, sentence):
        return self.komoran.pos(sentence)

    # 불용어 제거 후 필요한 품사 정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list

    # 키워드를 단어 인덱스 시퀀스로 변환
    def get_wordidx_sequence(self, keywords):
        if self.word_index is None:
            return []
        w2i = []
        for word in keywords:
            try:
                w2i.append(self.word_index[word])
            except KeyError:
                # 해당 단어가 사전에 없는 경우 OOV 처리
                w2i.append(self.word_index['OOV'])
        return w2i

p = Preprocess(word2index_dic='/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/love_chatbot_sbert_v2(custom_data)/data/chatbot_dict/chatbot_dict.bin',
               userdic=None)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing
import gc

# 의도 분류 모델 모듈
class IntentModel:
    def __init__(self, model_name, preprocess):

        # 의도 클래스별 레이블블
        self.labels = {0: "date/time/num_persons", 1: "date/time", 2: "date/num_persons"}

        # 의도 분류 모델 불러오기
        self.model = load_model(model_name)

        # 챗봇 텍스트 전처리기
        self.p = preprocess

    # 의도 클래스 예측
    def predict_class(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]





        # PADDING 최대 길이 설정
        # 단어 시퀀스 벡터 크기
        MAX_SEQ_LEN = 9

        # 패딩처리
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

        predict = self.model.predict(padded_seqs)
        predict_class = tf.math.argmax(predict, axis=1)
        return predict_class.numpy()[0]

In [ ]:
%cd /content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/tree_chatbot

# p는 이미 만듬
# p = Preprocess(word2index_dic='../train_tools/dict/chatbot_dict.bin',
#                userdic='../utils/user_dic.tsv')

reserve_info_classify_model = IntentModel(model_name='./models/CNN_reserve_info/sample_reserve_info_model.h5', preprocess=p)

# query = "내일 오후 2시에 8명 테이블 예약할게요.	"
# predict = intent.predict_class(query)
# predict_label = intent.labels[predict]
# print("="*30)
# print(query)
# print("의도 예측 클래스 : ", predict)
# print("의도 예측 레이블 : ", predict_label)

# 모델별 데이터 로딩

In [ ]:
# 의도 분류 모델의 데이터

n1_model_data = pd.read_excel("./data/csv/n1_model_data_embedding.xlsx")
n1_model_data

In [ ]:
# 예약 정보 확인 모델의 데이터

n2_model_data = pd.read_excel("./data/csv/n2_model_data_embedding.xlsx")
n2_model_data

In [ ]:
# 재예약 여부 확인 모델의 데이터

n3_model_data = pd.read_excel("./data/csv/n3_model_data_embedding.xlsx")
n3_model_data

In [ ]:
# 의도 분류 모델의 임베딩 데이터

n1_model_embedding_data = torch.load('./data/embedding_data/n1_model_data_embedding_data.pt')
n1_model_embedding_data

In [ ]:
# 예약 정보 확인 모델의  임베딩 데이터

n2_model_embedding_data = torch.load('./data/embedding_data/n2_model_data_embedding_data.pt')
n2_model_embedding_data

In [ ]:
# 재예약 여부 확인 모델의 임베딩 데이터

n3_model_embedding_data = torch.load('./data/embedding_data/n3_model_data_embedding_data.pt')
n3_model_embedding_data

# 트리 , 노드 정의

In [ ]:
# Getter, Setter 추가
# 클래스 변수 추가

class Tree:
  
  def __init__(self):
    self.root = None

  def set_root(self,node):
    self.root = node

  def get_root(self):
    return self.root

class Node:

  def __init__(self,info):

    # 설명
    self.info = info

    # 데이터 (예약정보:날짜/시간/명수 or 예약 가능 여부)
    self.data = {}

    # 각 노드별 역할(task) 구분 키
    # 기본 key = -1
    self.task_key = -1

    # 자식 노드들
    self.children = []

  def get_info(self):
    return self.info

  def set_info(self,new_info):
    self.info = new_info

  def get_task_key(self):
    return self.task_key

  def set_task_key(self,new_key):
    self.task_key = new_key

  def get_children(self):
    return self.children
  
  def set_children(self,*children):
    for child in children:
      self.children.append(child)

  def get_data(self):
    return self.data

  def set_data(self,dict_datum):
    self.data = dict_datum

  # 각 Node별 역할 
  def task(self,task_key,node):
    
    next_node = None

    if task_key == 1:

      print(f"챗봇: 안녕하세요. 신참떡볶이 AI 통화 비서입니다.\n 현재 통화 내용을 보다 더 나은 서비스 개선을 위해 수집될 수 있습니다. \n 무엇을 도와드릴까요? \n")
      user_input = str(input("사용자 : "))
      next_node = intent_classify(user_input,node)

    elif task_key == 2:
      print("챗봇: 네, 고객님 예약하시려는 날짜와 시간, 인원수를 말씀해주겠어요?\n")

      # 현재 노드(node)의 자식노드들(children) 인덱스
      # 왼->오 순 node들의 인덱스 0,1,2..
      # 인덱스 초기화
      idx = -1

      check_date = False
      check_time = False
      check_num_persons = False

      # 예약 정보 dict
      # 11월1일,1월11일 둘 다 111이므로 date는 str으로 
      dict_reserve_info = {"date" : "" , "time" : -1, "num_persons":-1}

      user_input = str(input("사용자 : "))
      idx , check_date , check_time , check_num_persons = reserve_info_classify(user_input,node)
        # print(check_date , check_time , check_num_persons)

      # 날짜 / 시간 / 인원 정보 모두 확인 후 다음 노드로
      print("<System> DB에서 검색중")
      # print("idx : " + str(idx))
      # check_date = False
      # check_time = False
      # check_num_persons = False
      # print(check_date , check_time , check_num_persons)

      next_node = node.children[idx]

      # 얻은 예약 정보들 다음 노드에
      # next_node.set_data(dict_reserve_info)

    return next_node


# 모델 별 Method

In [ ]:
# 예약 의도 분류 task(Method)

def intent_classify(input_intent,node):

  intent_thresholds = 0.7
  idx = -1
  
  sentence = input_intent.replace(" ","")
  sentence_encode = model1.encode(input_intent)
  sentence_tensor = torch.tensor(sentence_encode)

  cos_sim = util.cos_sim(sentence_tensor, n1_model_embedding_data)

  idx+=1
  if cos_sim[0][int(np.argmax(cos_sim))] < intent_thresholds:
    
    print("<System> 의도 분류 : 예약 X \n")

    return node.children[idx]

  idx+=1
  if cos_sim[0][int(np.argmax(cos_sim))] >= intent_thresholds:
    print("<System> 의도 분류 : 예약 O \n")

    best_sim_idx = int(np.argmax(cos_sim))
    answer = n1_model_data['output'][best_sim_idx]
    
    # print(f'사용자의 질문: {sentence} \n')
    print(f'<System> util.cos_sim 활용 코사인 유사도 : {cos_sim[0][best_sim_idx]}\n')

    print(f"챗봇 : {answer}\n")

    return node.children[idx]

In [ ]:
# 예약 정보 확인 task(Method)

def reserve_info_classify(input_reserve_info,node):

  global check_date 
  global check_time
  global check_num_persons

  global dict_reserve_info
  
  # 예약 정보 CNN 분류
  predict = reserve_info_classify_model.predict_class(input_reserve_info)
  predict_label = reserve_info_classify_model.labels[predict]

  print("<System> 예약 정보 예측 클래스 : ", predict, "\n") # 0, 1 ,2
  print("<System> 예약 정보 예측 레이블 : ", predict_label, "\n") # date/time/num_persons

  # 분류별 대응
  # label
  # 0 : 날짜 / 시간 / 인원  
  # 1 : 날짜 / 시간
  # 2 : 날짜 / 인원

  # 추가필요)
  # 3 : 인원 (인원만 재요청 시)
  # 4 : 시간 (시간만 재요청 시)


  # 확인한 정보 체크
  # 날짜 : 10 
  # 시간 : 11
  # 인원 : 12

  # 자식노드 인덱스
  idx = -1

  idx+=1
  # idx = 0
  if predict == idx:
    print("<System> 날짜 / 시간 / 인원 정보 모두 확인\n")
    print("<System> 정규식 활용 정보 추출\n")
    print("챗봇: 네, 잠시만요. (DB연동 예약 가능 여부 확인)\n")

    # dict_reserve_info["date"] = str(mmdd)
    # dict_reserve_info["time"] = n 시
    # dict_reserve_info["num_persons"] = n 명 

    check_date = True
    check_time = True
    check_num_persons = True

    return idx , check_date , check_time , check_num_persons #, dict_reserve_info
  
  idx+=1
  # idx = 1
  if predict == idx:
    print("<System> 날짜 / 시간 정보 확인\n")
    print("<System> 정규식 활용 정보 추출\n")    
    print("<System> 인원 정보 재요청\n")
    print("챗봇: 앗 인원은 몇분 정도 되실까요?\n")

    # dict_reserve_info["date"] = str(mmdd)
    # dict_reserve_info["time"] = n 시

    check_date = True
    check_time = True

    return idx , check_date , check_time , check_num_persons #, dict_reserve_info

  idx+=1
  # idx = 2
  if predict == idx:
    print("<System> 날짜 / 인원 정보 확인\n")
    print("<System> 정규식 활용 정보 추출\n")
    print("<System> 시간 정보 재요청\n")
    print("챗봇: 앗 몇시에 오실 예정이 신가요?\n")

    # dict_reserve_info["date"] = str(mmdd)
    # dict_reserve_info["num_persons"] = n 명 

    check_date = True
    check_num_persons = True
  
    return idx , check_date , check_time , check_num_persons #, dict_reserve_info

  idx+=1
  # idx = 3
  if predict == idx:
    print("<System> 날짜 / 시간 정보 + 인원 정보 확인\n")
    print("<System> 정규식 활용 정보 추출\n")
    print("챗봇: 네, 잠시만요. 확인해 드릴께요. (DB연동 예약 가능 여부 확인)\n")

    # dict_reserve_info["num_persons"] = n 명 

    check_num_persons = True

    return idx , check_date , check_time , check_num_persons #, dict_reserve_info

  idx+=1
  # idx = 4
  if predict == idx:
    print("<System> 날짜 / 인원 정보 + 시간 정보 확인\n")
    print("<System> 정규식 활용 정보 추출\n")
    print("챗봇: 네, 잠시만요. (DB연동 예약 가능 여부 확인)\n")

    # dict_reserve_info["time"] = n 시

    check_time = True

    return idx , check_date , check_time , check_num_persons #, dict_reserve_info

# 챗봇 실행 코드

In [ ]:
# 전위 호출
def work(node):

  # 현재 Node 출력
  print("<System> " + node.get_info() + "\n")

  while node != n7 or node != n10:

    print("\n")
    key = node.task_key
    nxt_node = node.task(key,node)
    work(nxt_node)

  if node == None:
    print("work()가 node 못받음")
  
  # 예약 완료
  if node == n7:
    print("챗봇: 예약이 완료되었습니다. 감사합니다.")
    return

  # 예약 취소
  if node == n10:
    print("챗봇: 다음에 뵙겠습니다. 감사합니다.")
    return

# 노드 세팅 및 실행

In [ ]:
### 노드 역할 ###

tree = Tree()
n0 = Node("Node0 : 예약 외의 업무 불가")
n1 = Node("Node1 : 의도 분류 단계")
n2 = Node("Node2 : 예약 정보(날짜/시간/인원) 요청 단계")
n3 = Node("Node3 : 날짜/인원 모두 확인")
n4 = Node("Node4 : 날짜 확인, 인원 미확인 => n2 재요청")
n5 = Node("Node5 : 인원 확인, 날짜 미확인 => n2 재요청")
n6 = Node("예약 가능 여부 체크")
n7 = Node("예약 정보 최종 확인 <종료>")
n8 = Node("예약 불가, 재예약 여부 요청")
n9 = Node("재예약 여부 확인 => n2 재요청")
n10 = Node("예약 취소 확인")
tree.set_root(n1)



### 노드 연결 ###

# 의도: 예약
n1.set_children(n0,n2)
# n1.children.append(n0)
# n1.children.append(n2)

# 예약 정보 확인
n2.set_children(n3,n4,n5)
# n2.children.append(n3)
# n2.children.append(n4)
# n2.children.append(n5)

# 예약 정보 확인 완료 - 예약 가능 여부 확인
n3.set_children(n6)
# n3.children.append(n6)

# 예약 가능 여부 확인
n6.set_children(n7,n8)
# n6.children.append(n7)
# n6.children.append(n8)

# 재예약 여부 확인
n8.set_children(n9,n10)
# n8.children.append(n9)
# n8.children.append(n10)

# 재예약 요청
n9.set_children(n6)
# n9.children.append(n6)



### 노드 task_key 설정 ###
n1.set_task_key(1)
n2.set_task_key(2)


###  사용자 input ###
# 사용자 통화 시 input 는 STT 잘 받았다고 가정
# input_intent = "저 예약좀 하려고요"
# input_reserve_info = "이번주 삼월일일 열두시에 다섯명이요" , "4월 5일로 3명 예약해주시겠어요?"
# input_again_reserve = "그럼 그 다음날 예약 가능할까요?"



### 챗봇 실행 ###

check_date = False
check_time = False
check_num_persons = False

work(tree.get_root())

# 연습

In [ ]:
class A:

  def __init__(self):
    self.data = {}
  
  def get_data(self):
    return self.data

  def set_data(self,dict_datum):
    self.data = dict_datum

In [ ]:
dict_datum = {1:"111", 2:"222"}

a = A()

In [ ]:
a.get_data()

In [ ]:
a.set_data(dict_datum)
a.data

In [ ]:
a.data

{}

In [ ]:
b = 1

flag = True

while flag:
  b = 2
  flag = False
b

2

In [ ]:
a = 1
 
def test():
    global a
    a = 3
    b = 2
 
    return a + b
 
 
print(test())
print(a)

5
3


In [ ]:
  if node == n0:
    print("챗봇: 현재 예약 이외의 문의는 어렵습니다.\n")